## 외래키 테이블

- 외래키 테이블 작성 요령

```SQL
DROP TABLE IF EXISTS teacher;
DROP TABLE IF EXISTS subject;

CREATE TABLE subject (
	sub_id   NUMERIC(4) NOT NULL PRIMARY KEY, 
	sub_name VARCHAR(60) NOT NULL
); 

INSERT INTO subject VALUES 
(0001, 'Mathematics'), 
(0002, 'History'), 
(0003, 'IT');

CREATE TABLE teacher (
	teac_id    NUMERIC(4) NOT NULL PRIMARY KEY, 
	teac_name  VARCHAR(20) NOT NULL, 
	sub_id     NUMERIC(5) REFERENCES subject, 
	hired_date DATE
); 

INSERT INTO teacher values (0011, '정선생', 0001, '2017-03-11'); -- 성공
INSERT INTO teacher values (0021, '박선생', 0004, '2017-05-20'); -- 실패
```

```
---

알림:  "teacher" 테이블 없음, 무시함
알림:  "subject" 테이블 없음, 무시함

ERROR: 오류:  "teacher" 테이블에서 자료 추가, 갱신 작업이 "teacher_sub_id_fkey" 참조키(foreign key) 제약 조건을 위배했습니다
DETAIL:  (sub_id)=(4) 키가 "subject" 테이블에 없습니다.


SQL state: 23503
```

- 외래키가 여러개인 경우 

```SQL
DROP TABLE IF EXISTS teacher;
DROP TABLE IF EXISTS subject;

CREATE TABLE subject (
	sub_id   NUMERIC(4) NOT NULL PRIMARY KEY, 
	sub_name VARCHAR(60) NOT NULL,
	UNIQUE (sub_id, sub_name)
); 

INSERT INTO subject VALUES 
(0001, 'Mathematics'), 
(0002, 'History'), 
(0003, 'IT');

CREATE TABLE teacher (
	teac_id    NUMERIC(4) NOT NULL PRIMARY KEY, 
	teac_name  VARCHAR(20) NOT NULL, 
	sub_code     NUMERIC(4) NOT NULL, 
	sub_name   VARCHAR(60) NOT NULL, 
	hired_date DATE,  
	FOREIGN KEY (sub_code, sub_name) REFERENCES subject (sub_id, sub_name)
); 

INSERT INTO teacher values (0011, '정선생', 0003, 'IT', '2017-03-11'); -- 성공
INSERT INTO teacher values (0021, '박선생', 0004, 'IT', '2017-05-20'); -- 실패
```

## ON DELETE
- ON DELETE RESTRICT, 과목 테이블 및 교사 정보 테이블

```SQL
DROP TABLE IF EXISTS teacher;
DROP TABLE IF EXISTS subject;

CREATE TABLE subject (
	sub_id   NUMERIC(4) NOT NULL PRIMARY KEY, 
	sub_name VARCHAR(60) NOT NULL
); 

INSERT INTO subject VALUES 
(0001, 'Mathematics'), 
(0002, 'History'), 
(0003, 'IT');

CREATE TABLE teacher (
	teac_id   NUMERIC(4) NOT NULL PRIMARY KEY, 
	teac_name VARCHAR(20) NOT NULL, 
	sub_id    NUMERIC(4) REFERENCES subject ON DELETE RESTRICT, 
	hire_date DATE
); 

INSERT INTO teacher values (0011, '정선생', 0001, '2017-03-11'); -- 성공
INSERT INTO teacher values (0021, '박선생', 0002, '2017-05-20'); -- 성공
INSERT INTO teacher values (0031, '김선생', 0003, '2017-04-13'); -- 성공

DELETE FROM subject WHERE sub_id = 0002;
```
```
ERROR:  update or delete on table "subject" violates foreign key constraint "teacher_sub_id_fkey" on table "teacher"
DETAIL:  Key (sub_id)=(2) is still referenced from table "teacher".
SQL state: 23503
```

- CASCADE 조건

```SQL
DROP TABLE IF EXISTS teacher;
DROP TABLE IF EXISTS subject;

CREATE TABLE subject (
	sub_id   NUMERIC(4) NOT NULL PRIMARY KEY, 
	sub_name VARCHAR(60) NOT NULL
); 

INSERT INTO subject VALUES 
(0001, 'Mathematics'), 
(0002, 'History'), 
(0003, 'IT');

CREATE TABLE teacher (
	teac_id   NUMERIC(4) NOT NULL PRIMARY KEY, 
	teac_name VARCHAR(20) NOT NULL, 
	sub_id    NUMERIC(4) REFERENCES subject ON DELETE CASCADE, 
	hire_date DATE
); 

INSERT INTO teacher values (0011, '정선생', 0001, '2017-03-11'); -- 성공
INSERT INTO teacher values (0021, '박선생', 0002, '2017-05-20'); -- 성공
INSERT INTO teacher values (0031, '김선생', 0003, '2017-04-13'); -- 성공
 teac_id | teac_name | sub_id | hire_date  
---------+-----------+--------+------------
      11 | 정선생    |      1 | 2017-03-11
      21 | 박선생    |      2 | 2017-05-20
      31 | 김선생    |      3 | 2017-04-13
(3 rows)
```

- 참조된 값을 지운 다음, 다시 테이블을 불러온다. 그러면 이번에는 정상적으로 지워지는 것을 확인할 수 있다. 
- ON DELETE CASCADE 조건은 참조한 열을 모두 지운다. 

```SQL
postgres=# DELETE FROM subject WHERE sub_id = 0002;
DELETE 1
postgres=# SELECT * FROM subject;
 sub_id |  sub_name   
--------+-------------
      1 | Mathematics
      3 | IT
(2 rows)
postgres=# SELECT * FROM teacher;
 teac_id | teac_name | sub_id | hire_date  
---------+-----------+--------+------------
      11 | 정선생    |      1 | 2017-03-11
      31 | 김선생    |      3 | 2017-04-13
(2 rows)
```

- 이번에는 ON DELETE SET NULL 조건을 배운다. 

```SQL
DROP TABLE IF EXISTS teacher;
DROP TABLE IF EXISTS subject;

CREATE TABLE subject (
	sub_id   NUMERIC(4) NOT NULL PRIMARY KEY, 
	sub_name VARCHAR(60) NOT NULL
); 

INSERT INTO subject VALUES 
(0001, 'Mathematics'), 
(0002, 'History'), 
(0003, 'IT');

CREATE TABLE teacher (
	teac_id   NUMERIC(4) NOT NULL PRIMARY KEY, 
	teac_name VARCHAR(20) NOT NULL, 
	sub_id    NUMERIC(4) REFERENCES subject ON DELETE SET NULL, 
	hire_date DATE
); 

INSERT INTO teacher values (0011, '정선생', 0001, '2017-03-11'); -- 성공
INSERT INTO teacher values (0021, '박선생', 0002, '2017-05-20'); -- 성공
INSERT INTO teacher values (0031, '김선생', 0003, '2017-04-13'); -- 성공

postgres=# DELETE FROM subject WHERE sub_id = 0002;
DELETE 1
postgres=# SELECT * FROM subject;
 sub_id |  sub_name   
--------+-------------
      1 | Mathematics
      3 | IT
(2 rows)

postgres=# SELECT * FROM teacher;
 teac_id | teac_name | sub_id | hire_date  
---------+-----------+--------+------------
      11 | 정선생    |      1 | 2017-03-11
      31 | 김선생    |      3 | 2017-04-13
      21 | 박선생    |        | 2017-05-20
(3 rows)
```

- 이번에는 ON DELETE SET DEFAULT 조건을 입력하도록 합니다. 

```SQL
DROP TABLE IF EXISTS teacher;
DROP TABLE IF EXISTS subject;

CREATE TABLE subject (
	sub_id   NUMERIC(4) NOT NULL PRIMARY KEY, 
	sub_name VARCHAR(60) NOT NULL
); 

INSERT INTO subject VALUES 
(0001, 'Mathematics'), 
(0002, 'History'), 
(0003, 'IT');

CREATE TABLE teacher (
	teac_id   NUMERIC(4) NOT NULL PRIMARY KEY, 
	teac_name VARCHAR(20) NOT NULL, 
	sub_id    NUMERIC(4) DEFAULT 1 REFERENCES subject ON DELETE SET DEFAULT, 
	hire_date DATE
); 

INSERT INTO teacher values (0011, '정선생', 0001, '2017-03-11'); -- 성공
INSERT INTO teacher values (0021, '박선생', 0002, '2017-05-20'); -- 성공
INSERT INTO teacher values (0031, '김선생', 0003, '2017-04-13'); -- 성공

postgres=# DELETE FROM subject WHERE sub_id = 0002;
DELETE 1
postgres=# SELECT * FROM subject;
 sub_id |  sub_name   
--------+-------------
      1 | Mathematics
      3 | IT
(2 rows)

postgres=# SELECT * FROM teacher;
 teac_id | teac_name | sub_id | hire_date  
---------+-----------+--------+------------
      11 | 정선생    |      1 | 2017-03-11
      31 | 김선생    |      3 | 2017-04-13
      21 | 박선생    |      1 | 2017-05-20
(3 rows)
```

- Check 제약 조건이 담긴 테이블을 생성한다. 

```SQL
DROP TABLE IF EXISTS order_info;

CREATE TABLE order_info(
	order_no INTEGER NOT NULL PRIMARY KEY, 
	cust_name VARCHAR(100), 
	price MONEY, 
	order_qty INTEGER CHECK (order_qty > 0) -- 양수만을 입력받기로 한다. 
); 

INSERT INTO order_info values (1, '김선생', 3000, 2); -- 성공
INSERT INTO order_info values (2, '홍선생', 4000, -2); -- 실패

ERROR:  new row for relation "order_info" violates check constraint "order_info_order_qty_check"
DETAIL:  Failing row contains (2, 홍선생, $4,000.00, -2).
SQL state: 23514
```